# RQ2

#### [RQ2] What are the time slots with more passengers? Set your own time slots and discover which are those when Taxis drive the highest number of passengers overall New York and repeat the analysis for each borough. Provide the results through a visualization and comment them.

In [132]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

%matplotlib inline


import matplotlib.pyplot as plt

from sys import getsizeof as get

In [82]:
#We choose the departure boroughs. 
#We know prior which columns of the dataset we need so by using usecols while reading .csv files, try to save some memory

In [133]:
#df = pd.read_csv(r"C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-01.csv", usecols = ['tpep_pickup_datetime','passenger_count', 'PULocationID'])
#df = pd.read_csv(r"C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-02.csv", usecols = ['tpep_pickup_datetime','passenger_count', 'PULocationID'])
#df = pd.read_csv(r"C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-03.csv", usecols = ['tpep_pickup_datetime','passenger_count', 'PULocationID'])
#df = pd.read_csv(r"C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-04.csv", usecols = ['tpep_pickup_datetime','passenger_count', 'PULocationID'])
df = pd.read_csv(r"C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-05.csv", usecols = ['tpep_pickup_datetime','passenger_count', 'PULocationID'])
#df = pd.read_csv(r"C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-06.csv", usecols = ['tpep_pickup_datetime','passenger_count', 'PULocationID'])

In [134]:
df.head()

,tpep_pickup_datetime,passenger_count,PULocationID
0,2018-05-01 00:13:56,1,230
1,2018-05-01 00:23:26,1,263
2,2018-05-01 00:36:23,2,239
3,2018-05-01 00:26:12,1,145
4,2018-05-01 00:29:51,1,145


In [135]:
df.shape

(9224063, 3)

In [136]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9224063 entries, 0 to 9224062
Data columns (total 3 columns):
tpep_pickup_datetime    object
passenger_count         int64
PULocationID            int64
dtypes: int64(2), object(1)
memory usage: 809.3 MB


# Data Cleaning

In [87]:
#We have already done a bit of data cleaning
#At the time of loading the datasset, by filtering these columns:['tpep_pickup_datetime', 'passenger_count', 'PULocationID']
#We know the contet of the dataset prior, so we keep those columns that we will use for this certain question

In [137]:
#Check if there are any NaN values
df.isnull().any().any()

False

In [138]:
#'tpep_pickup_datetime' is of Object type. we need to convert it into datetime. This saves memory(average memory usage of an object type is 9.5 mb)
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

In [90]:
#Now that we have a new column called 'datetime' with the type datetime64, we drop 'tpep_pickup_datetime'.
#df = df.drop(['tpep_pickup_datetime'], axis=1)

In [139]:
#We have two numeric collumns of subtype int64
#One is 'passenger_count'. This columns contains positve numbers and they are supposed to between 1 and 6
#The other column is the Zone numbers, 'PULocationID' which again are positive numbers between 1 and 265 (according to taxi_zone lookup table data)
#To save these numbers another integer suntyoe which uses less memory can be used too: uint8 and uint16.
#These two save 1 and 2 bytes respectively, compared to int64 that uses 8 bytes per number. 
df['passenger_count'] = df['passenger_count'].astype(np.uint8)
df['PULocationID'] = df['PULocationID'].astype(np.uint16)

In [140]:
df.info(memory_usage='deep')
#Object columns are used for strings or where a column contains mixed data types.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9224063 entries, 0 to 9224062
Data columns (total 3 columns):
tpep_pickup_datetime    datetime64[ns]
passenger_count         uint8
PULocationID            uint16
dtypes: datetime64[ns](1), uint16(1), uint8(1)
memory usage: 96.8 MB


### Datetime 

In [141]:
# extract year and month to check if each row contains the right values: year 2018 and month 1(January) 
# extract hour in order to create the time slots

df['Year'] = pd.DatetimeIndex(df['tpep_pickup_datetime']).year
df['Month'] = pd.DatetimeIndex(df['tpep_pickup_datetime']).month
df['Day'] = pd.DatetimeIndex(df['tpep_pickup_datetime']).day
df['Hour'] = pd.DatetimeIndex(df['tpep_pickup_datetime']).hour

In [142]:
df.head()

,tpep_pickup_datetime,passenger_count,PULocationID,Year,Month,Day,Hour
0,2018-05-01 00:13:56,1,230,2018,5,1,0
1,2018-05-01 00:23:26,1,263,2018,5,1,0
2,2018-05-01 00:36:23,2,239,2018,5,1,0
3,2018-05-01 00:26:12,1,145,2018,5,1,0
4,2018-05-01 00:29:51,1,145,2018,5,1,0


In [143]:
#Drop 'datetime'
#Data has been extracted out of it so we don't need this column anymore
df.drop(columns=['tpep_pickup_datetime'], inplace=True)

In [144]:
df.head()

,passenger_count,PULocationID,Year,Month,Day,Hour
0,1,230,2018,5,1,0
1,1,263,2018,5,1,0
2,2,239,2018,5,1,0
3,1,145,2018,5,1,0
4,1,145,2018,5,1,0


In [145]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9224063 entries, 0 to 9224062
Data columns (total 6 columns):
passenger_count    uint8
PULocationID       uint16
Year               int64
Month              int64
Day                int64
Hour               int64
dtypes: int64(4), uint16(1), uint8(1)
memory usage: 307.9 MB


In [146]:
df['Year'] = df['Year'].astype(np.uint16)
df['Month'] = df['Month'].astype(np.uint8)
df['Day'] = df['Day'].astype(np.uint8)
df['Hour'] = df['Hour'].astype(np.uint8)

In [147]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9224063 entries, 0 to 9224062
Data columns (total 6 columns):
passenger_count    uint8
PULocationID       uint16
Year               uint16
Month              uint8
Day                uint8
Hour               uint8
dtypes: uint16(2), uint8(4)
memory usage: 70.4 MB


In [148]:
#Doing some data cleaning: checking to see if year=2018, Month=1, 1<=day<=31 and 0<=hour<=23
#We use groupby function to see the groups in each of these columns

#df.groupby('Year').count()
df.Year.unique()

,passenger_count,PULocationID,Month,Day,Hour
Year,,,,,
2008,28,28,28,28,28
2009,35,35,35,35,35
2018,9223999,9223999,9223999,9223999,9223999
2041,1,1,1,1,1


In [101]:
#del df

In [149]:
#Drop those rows containing years other than 2018
#df.drop(df[(df.Year == 2017) | (df.Year == 2009) | (df.Year == 2008) | (df.Year == 2003) | (df.Year == 2002) | (df.Year == 2001)].index, inplace=True)
#df.drop(df[(df.Year == 2008) | (df.Year == 2009) | (df.Year == 2019) | (df.Year == 2026) | (df.Year == 2029) | (df.Year == 2031)].index, inplace=True)
#df.drop(df[(df.Year == 2001) | (df.Year == 2002) | (df.Year == 2003) | (df.Year == 2008) | (df.Year == 2009) | (df.Year == 2019) | (df.Year == 2037)| (df.Year == 2042)| (df.Year == 2084)| (df.Year == 2020) ].index, inplace=True)
#df.drop(df[(df.Year == 2001) | (df.Year == 2008) | (df.Year == 2029) | (df.Year == 2009)].index, inplace=True)
df.drop(df[(df.Year == 2008) | (df.Year == 2009) | (df.Year == 2041)].index, inplace=True)
df.shape

(9223999, 6)

In [150]:
#Checking if the Month column is clean aka Month=1
#df.groupby('Month').count()
df.Month.unique()

,passenger_count,PULocationID,Year,Day,Hour
Month,,,,,
4,186,186,186,186,186
5,9223745,9223745,9223745,9223745,9223745
6,68,68,68,68,68


In [151]:
#Drop those rows containing Months other than Dataset1: January(1), Dataset2: Feburary(2), Dataset3: March(3), Dataset4: April(4), Dataset5: May(5), Dataset6: June(6)
#df.drop(df[(sdf.Month == 2) | (df.Month == 7)].index)
#df.drop(df[(sdf.Month == 1) | (df.Month == 3) | (df.Month == 4)].index)
#df.drop(df[(sdf.Month == 2) | (df.Month == 4) | (df.Month == 5) | (df.Month == 6) | (df.Month == 7) | (df.Month == 8) | (df.Month == 9) | (df.Month == 10) | (df.Month == 11) | (df.Month == 12)].index)
#df.drop(df[(df.Month == 3) | (df.Month == 5) | (df.Month == 6) | (df.Month == 7) | (df.Month == 8) | (df.Month == 9)].index)
df.drop(df[(df.Month == 4) | (df.Month == 6)].index, inplace=True )
df.shape

(9223745, 6)

In [ ]:
#Check the Day column
#df.groupby('Day').count()
sorted(df['Day'].unique())

In [179]:
#Day column seems alright
#We check the hour column
#df.groupby('Hour').count()
sorted(df.Hour.unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [108]:
#Hour column seems fine too (for datasets 1 and 2)

In [173]:
#Now based on our Research question in hand(RQ2), we won't need these columns: Year, Month and Day.
#We drop them
df.drop(['Month', 'Day', 'Year'], axis=1, inplace=True)

In [174]:
df.head()

,passenger_count,PULocationID,Hour
0,1,230,0
1,1,263,0
2,2,239,0
3,1,145,0
4,1,145,0


In [181]:
#passenger_count gives us the number of passenger in each car
#We go about cleaning this column based on this information presented in the yellow taxi's website:
#The maximum number of passengers allowed in a yellow taxicab by law is four (4) in a four (4) passenger taxicab or
#five (5) passengers in a five (5) passenger taxicab, 
#except that an additional passenger must be accepted if such passenger is under the age of seven (7) and 
#is held on the lap of an adult passenger seated in the rear."
#So, we see that 1<=passenger count<=6

df.groupby('passenger_count').count()
#sorted(df['passenger_count'].unique())


,PULocationID,Hour
passenger_count,,
0,72081,72081
1,6580753,6580753
2,1348201,1348201
3,372369,372369
4,173485,173485
5,421964,421964
6,254800,254800
7,33,33
8,33,33


In [183]:
#We drop those rows with values >=7 and 0
#df.drop(df[(df.passenger_count == 7) | (df.passenger_count == 8) | (df.passenger_count == 9) | (df.passenger_count == 0)].index, inplace=True)
#df.drop(df[(df.passenger_count == 7) | (df.passenger_count == 8) | (df.passenger_count == 9) | (df.passenger_count == 0) | (df.passenger_count == 192)].index, inplace=True)
#df.drop(df[(df.passenger_count == 7) | (df.passenger_count == 8) | (df.passenger_count == 9) | (df.passenger_count == 0)].index, inplace=True)
#df.drop(df[(df.passenger_count == 7) | (df.passenger_count == 8) | (df.passenger_count == 9) | (df.passenger_count == 0)].index, inplace=True)
df.drop(df[(df.passenger_count == 0) | (df.passenger_count == 7) | (df.passenger_count == 8) | (df.passenger_count == 9)].index, inplace=True)
df.shape

(9151572, 3)

#### Merge two datasets

In [184]:
#Read taxi_zone_lookup.csv
df2= pd.read_csv(r"C:\Users\samin\Desktop\Homework2\taxi_zone_lookup.csv", usecols=['LocationID', 'Borough'])

In [185]:
df2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 2 columns):
LocationID    265 non-null int64
Borough       265 non-null object
dtypes: int64(1), object(1)
memory usage: 18.9 KB


In [186]:
df2.head()

,LocationID,Borough
0,1,EWR
1,2,Queens
2,3,Bronx
3,4,Manhattan
4,5,Staten Island


In [187]:
#Change the type of Borough column 
#df2['Borough'] = df2['Borough'].astype('|S') #by default set the length to the max len it encounters

In [191]:
df2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 2 columns):
LocationID    265 non-null int64
Borough       265 non-null object
dtypes: int64(1), object(1)
memory usage: 14.7 KB


In [192]:
df2.head()

,LocationID,Borough
0,1,b'EWR'
1,2,b'Queens'
2,3,b'Bronx'
3,4,b'Manhattan'
4,5,b'Staten Island'


In [ ]:
#In sdf dataframe, we have a column that contains the number of zones in each borough. 
#In order to get the name of the boroughs, we need to merge sdf with another dataset available (taxi_zone_lookup.csv)
#We read this dataset with pd.read_csv() and called the dataframe borough

In [193]:
result = pd.merge(df,
                 df2,
                 left_on='PULocationID',
                 right_on='LocationID',
                 how='inner' 
                 )
result.head()
del df

In [194]:
#Drop 'LocationID' and 'PULocationID'
result.drop(['PULocationID', 'LocationID'], axis =1, inplace=True)

In [195]:
result.head()

,passenger_count,Hour,Borough
0,1,0,b'Manhattan'
1,1,0,b'Manhattan'
2,1,0,b'Manhattan'
3,1,0,b'Manhattan'
4,5,0,b'Manhattan'


In [122]:
result.describe()

,passenger_count,Hour
count,9.232099e+06,9.232099e+06
mean,1.612919e+00,1.383333e+01
std,1.242232e+00,6.112564e+00
min,1.000000e+00,0.000000e+00
25%,1.000000e+00,1.000000e+01
50%,1.000000e+00,1.400000e+01
75%,2.000000e+00,1.900000e+01
max,6.000000e+00,2.300000e+01


In [123]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9232099 entries, 0 to 9232098
Data columns (total 3 columns):
passenger_count    uint8
Hour               uint8
Borough            object
dtypes: object(1), uint8(2)
memory usage: 158.5+ MB


In [ ]:
#Checking the Boroughs
#In NYC, there are six boroughs: Bronx, Brooklyn, Manhattan, Staten Island, Queens

In [124]:
result.groupby('Borough').count()

,passenger_count,Hour
Borough,,
Bronx,10302,10302
Brooklyn,118221,118221
EWR,683,683
Manhattan,8365587,8365587
Queens,591696,591696
Staten Island,128,128
Unknown,145482,145482


In [125]:
#We see two unexpected names in Broughs category: EWR which an airport near Queens and Unknown
#We drop the rows with borough=Unknown or borough=EWR
result.drop(result[(result.Borough == 'EWR') | (result.Borough == 'Unknown')].index, inplace=True)
result.shape

(9085934, 3)

#### Defining Time-slots

In [126]:
result.head()

,passenger_count,Hour,Borough
0,1,0,Queens
1,1,0,Queens
2,2,0,Queens
3,3,0,Queens
4,1,0,Queens


In [127]:
#To define a time slot (and probably labeling it) we use pandas function .cut (pd.cut())
result=result.assign(
    timeSlot=pd.cut(
        result['Hour'],
        [-1, 8, 12, 18, 20, 23],
        labels=['0-8', '8-12', '12-18', '18-20', '20-23']))

result.drop(['Hour'], axis=1, inplace=True)

In [128]:
result.groupby(['timeSlot', 'Borough']).sum()

passenger_count
timeSlot Borough                       
0-8      Bronx                   5988.0
         Brooklyn               67046.0
         Manhattan            2710028.0
         Queens                166644.0
         Staten Island             73.0
8-12     Bronx                   3516.0
         Brooklyn               24802.0
         Manhattan            2600473.0
         Queens                170469.0
         Staten Island             31.0
12-18    Bronx                   4608.0
         Brooklyn               42176.0
         Manhattan            4462346.0
         Queens                346619.0
         Staten Island             47.0
18-20    Bronx                   1149.0
         Brooklyn               18205.0
         Manhattan            1654621.0
         Queens                118888.0
         Staten Island             13.0
20-23    Bronx                   1622.0
         Brooklyn               38011.0
         Manhattan            2073633.0
         Queens                167682.0
         Staten Island             32.0

In [ ]:
result.head()

### Save the result dataframes for each month for the final plotting

In [129]:
#Save result dataframe as a .csv file


#result.to_csv(r'C:\Users\samin\Desktop\Homework2\RQ2Jan.csv')
#result.to_csv(r'C:\Users\samin\Desktop\Homework2\RQ2Feb.csv')
#result.to_csv(r'C:\Users\samin\Desktop\Homework2\RQ2March.csv')
#result.to_csv(r'C:\Users\samin\Desktop\Homework2\RQ2Apr.csv')
result.to_csv(r'C:\Users\samin\Desktop\Homework2\RQ2May.csv')
#result.to_csv(r'C:\Users\samin\Desktop\Homework2\RQ2Jun.csv')


In [ ]:
#df=result.groupby(['timeSlot'])['passenger_count'].sum()

In [ ]:
#df.plot.bar()

In [ ]:
#df1 = result.groupby(['timeSlot', 'Borough'])['passenger_count'].sum()

In [ ]:
#df1.plot.bar(figsize=(15, 9))

In [131]:
del result